In [11]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from ipywidgets import interact, interactive, widget_color
from IPython.display import display

try:
    import seaborn as sns
except ImportError:
    print('Seaborn is not installed')
%matplotlib inline

from tyssue import Sheet, config
from tyssue import SheetGeometry as geom
from tyssue.core.generation import three_faces_sheet

from tyssue.draw.threejs_draw import view_3js



# Grab the default settings for drawing
draw_specs = config.draw.sheet_spec()

from matplotlib.colors import hex2color

### Create a sheet

In [2]:
datasets, specs = three_faces_sheet()
sheet = Sheet('3cells_2D', datasets, specs)
geom.update_all(sheet)

## Default 

In [6]:
renderer, line = view_3js(sheet)
display(renderer)

In [7]:
from tyssue.io.hdf5 import load_datasets
from tyssue.core.monolayer import Monolayer

In [8]:
dsets = load_datasets('../data/hf5/small_ellipsoid.hf5',
                      data_names=['vert', 'edge', 'face', 'cell'])
specs = config.geometry.bulk_spec()

mono = Monolayer('mono', datasets=dsets, specs=specs)

In [37]:
def set_colors(monolayer, apical_color, basal_color):

    colors = pd.DataFrame(np.zeros((monolayer.Nv, 3)),
                          index=monolayer.vert_df.index,
                          columns=['R', 'G', 'B'])
    for i, c in enumerate('RGB'):
        colors.loc[monolayer.apical_verts, c] = apical_color[i]
        colors.loc[monolayer.basal_verts, c] = basal_color[i]
    return colors

draw_specs['edge']['width'] = 1.5

## Interactive 3D view of the modeled epithelium


This [ThreeJS](https://threejs.org/) widget allows to display a 3D tissue within
your `jupyter notebook`.

Use the mouse wheel to zoom in or out, and turn around the object by dragging your pointer
in the widget's window.

You can pick the color of the apical and basal layers by clicking
in the squares bellow.

In [42]:
apical_cw = widget_color.ColorPicker(description='apical color')
basal_cw = widget_color.ColorPicker(description='basal color')

display(apical_cw, basal_cw)


In [45]:
apical_color = hex2color(apical_cw.value)
basal_color = hex2color(basal_cw.value)

draw_specs['vert']['color'] = set_colors(mono, apical_color,
                                         basal_color)

renderer, line = view_3js(mono, **draw_specs)

display(renderer)

In [10]:
import pythreejs as tjs

In [15]:
apical_sheet = mono.get_sub_sheet('apical')
apical_sheet.reset_index()
apical_sheet.reset_topo()
from tyssue.core.objects import ordered_vert_idxs
from matplotlib.colors import rgb2hex

vertices, faces, _ = apical_sheet.vertex_mesh(['x', 'y', 'z'])
face_colors = ['#aaee11' for f in faces]

geometry = tjs.FaceGeometry(vertices=list(vertices.flat),
                            facen=[list(f) for f in faces],
                            colors=face_colors)
material=tjs.LambertMaterial(color='green',
                             #vertexColors='FaceColors',
                             side='FrontSide')
material.transparent = True
material.opacity = 0.3
surf = tjs.Mesh(geometry=geometry, material=material)


scene = tjs.Scene(children=[surf,
                            tjs.DirectionalLight(color='#ffaa00',
                                                 position=[0, 5, 0]),
                            tjs.AmbientLight(color='#cccccc')])
c = tjs.PerspectiveCamera(position=[5, 5, 3], up=[0, 0, 1],
                          children=[tjs.DirectionalLight(color='red',
                                                         position=[3, 5, 1],
                                                         intensity=0.6)])
renderer = tjs.Renderer(camera=c, scene=scene, 
                        controls=[tjs.OrbitControls(controlling=c)])
display(renderer)